# 디스크 컨트롤

<div class="guide-section-description">
      <h6 class="guide-section-title">문제 설명</h6>
      <div class="markdown solarized-dark"><p>하드디스크는 한 번에 하나의 작업만 수행할 수 있습니다. 디스크 컨트롤러를 구현하는 방법은 여러 가지가 있습니다. 가장 일반적인 방법은 요청이 들어온 순서대로 처리하는 것입니다.</p>

<p>예를들어</p>
<div class="highlight"><pre class="codehilite"><code>- 0ms 시점에 3ms가 소요되는 A작업 요청
- 1ms 시점에 9ms가 소요되는 B작업 요청
- 2ms 시점에 6ms가 소요되는 C작업 요청
</code></pre></div>
<p>와 같은 요청이 들어왔습니다. 이를 그림으로 표현하면 아래와 같습니다.<br>
  <img src="https://grepp-programmers.s3.amazonaws.com/files/production/b68eb5cec6/38dc6a53-2d21-4c72-90ac-f059729c51d5.png" title="" alt="Screen Shot 2018-09-13 at 6.34.58 PM.png"></p>

<p>한 번에 하나의 요청만을 수행할 수 있기 때문에 각각의 작업을 요청받은 순서대로 처리하면 다음과 같이 처리 됩니다.<br>
  <img src="https://grepp-programmers.s3.amazonaws.com/files/production/5e677b4646/90b91fde-cac4-42c1-98b8-8f8431c52dcf.png" title="" alt="Screen Shot 2018-09-13 at 6.38.52 PM.png"></p>
<div class="highlight"><pre class="codehilite"><code>- A: 3ms 시점에 작업 완료 (요청에서 종료까지 : 3ms)
- B: 1ms부터 대기하다가, 3ms 시점에 작업을 시작해서 12ms 시점에 작업 완료(요청에서 종료까지 : 11ms)
- C: 2ms부터 대기하다가, 12ms 시점에 작업을 시작해서 18ms 시점에 작업 완료(요청에서 종료까지 : 16ms)
</code></pre></div>
<p>이 때 각 작업의 요청부터 종료까지 걸린 시간의 평균은 10ms(= (3 + 11 + 16) / 3)가 됩니다.</p>

<p>하지만 A → C → B 순서대로 처리하면<br>
  <img src="https://grepp-programmers.s3.amazonaws.com/files/production/9eb7c5a6f1/a6cff04d-86bb-4b5b-98bf-6359158940ac.png" title="" alt="Screen Shot 2018-09-13 at 6.41.42 PM.png"></p>
<div class="highlight"><pre class="codehilite"><code>- A: 3ms 시점에 작업 완료(요청에서 종료까지 : 3ms)
- C: 2ms부터 대기하다가, 3ms 시점에 작업을 시작해서 9ms 시점에 작업 완료(요청에서 종료까지 : 7ms)
- B: 1ms부터 대기하다가, 9ms 시점에 작업을 시작해서 18ms 시점에 작업 완료(요청에서 종료까지 : 17ms)
</code></pre></div>
<p>이렇게 A → C → B의 순서로 처리하면 각 작업의 요청부터 종료까지 걸린 시간의 평균은 9ms(= (3 + 7 + 17) / 3)가 됩니다.</p>

<p>각 작업에 대해 [작업이 요청되는 시점, 작업의 소요시간]을 담은 2차원 배열 jobs가 매개변수로 주어질 때, 작업의 요청부터 종료까지 걸린 시간의 평균을 가장 줄이는 방법으로 처리하면 평균이 얼마가 되는지 return 하도록 solution 함수를 작성해주세요. (단, 소수점 이하의 수는 버립니다)</p>

<h5>제한 사항</h5>

<ul>
<li>jobs의 길이는 1 이상 500 이하입니다.</li>
<li>jobs의 각 행은 하나의 작업에 대한 [작업이 요청되는 시점, 작업의 소요시간] 입니다.</li>
<li>각 작업에 대해 작업이 요청되는 시간은 0 이상 1,000 이하입니다.</li>
<li>각 작업에 대해 작업의 소요시간은 1 이상 1,000 이하입니다.</li>
<li>하드디스크가 작업을 수행하고 있지 않을 때에는 먼저 요청이 들어온 작업부터 처리합니다.</li>
</ul>

<h5>입출력 예</h5>
<table class="table">
        <thead><tr>
<th>jobs</th>
<th>return</th>
</tr>
</thead>
        <tbody><tr>
<td>[[0, 3], [1, 9], [2, 6]]</td>
<td>9</td>
</tr>
</tbody>
      </table>
<h5>입출력 예 설명</h5>

<p>문제에 주어진 예와 같습니다.</p>

<ul>
<li>0ms 시점에 3ms 걸리는 작업 요청이 들어옵니다.</li>
<li>1ms 시점에 9ms 걸리는 작업 요청이 들어옵니다.</li>
<li>2ms 시점에 6ms 걸리는 작업 요청이 들어옵니다.</li>
</ul>
</div>
    </div>

In [105]:
'''
1. 소요시간을 기준으로 정렬
2. 시간이 흐름에 따라 요청이 들어오는 작업의 우선순위를 매겨서 처리
    2-1. 대기목록을 요청길이로 정렬
    2-2. 대기열 맨 앞 작업을 채택
    2-3. 현재 시간을 새로운 요청시간만큼 증가
    2-4. 대기 시간(현재시간-작업시작시간)을 업데이트
    2-5. 전체 작업에서 방금했던 작업을 제거
3. 대기 시간의 평균을 리턴
    
'''

def solution(jobs):
    length = len(jobs)
    cur_time = 0
    answer = 0
    
    while jobs:
        waiting = [i for i in jobs if i[0] <= cur_time]
        if waiting:
            waiting.sort(key=lambda x: x[1])
            next_job = waiting[0]
            cur_time += next_job[1]
            answer += cur_time - next_job[0]
            jobs.remove(next_job)
        else:
            cur_time += 1

    return answer // length

jobs = [[0, 3], [1, 9], [2, 6]]
solution(jobs)

9

# 베스트 앨범

<div class="guide-section-description">
      <h6 class="guide-section-title">문제 설명</h6>
      <div class="markdown solarized-dark"><p>스트리밍 사이트에서 장르 별로 가장 많이 재생된 노래를 두 개씩 모아 베스트 앨범을 출시하려 합니다. 노래는 고유 번호로 구분하며, 노래를 수록하는 기준은 다음과 같습니다.</p>

<ol>
<li>속한 노래가 많이 재생된 장르를 먼저 수록합니다.</li>
<li>장르 내에서 많이 재생된 노래를 먼저 수록합니다.</li>
<li>장르 내에서 재생 횟수가 같은 노래 중에서는 고유 번호가 낮은 노래를 먼저 수록합니다.</li>
</ol>

<p>노래의 장르를 나타내는 문자열 배열 genres와 노래별 재생 횟수를 나타내는 정수 배열 plays가 주어질 때, 베스트 앨범에 들어갈 노래의 고유 번호를 순서대로 return 하도록 solution 함수를 완성하세요.</p>

<h5>제한사항</h5>

<ul>
<li>genres[i]는 고유번호가 i인 노래의 장르입니다.</li>
<li>plays[i]는 고유번호가 i인 노래가 재생된 횟수입니다.</li>
<li>genres와 plays의 길이는 같으며, 이는 1 이상 10,000 이하입니다.</li>
<li>장르 종류는 100개 미만입니다.</li>
<li>장르에 속한 곡이 하나라면, 하나의 곡만 선택합니다.</li>
<li>모든 장르는 재생된 횟수가 다릅니다.</li>
</ul>

<h5>입출력 예</h5>
<table class="table">
        <thead><tr>
<th>genres</th>
<th>plays</th>
<th>return</th>
</tr>
</thead>
        <tbody><tr>
<td>[<q>classic</q>, <q>pop</q>, <q>classic</q>, <q>classic</q>, <q>pop</q>]</td>
<td>[500, 600, 150, 800, 2500]</td>
<td>[4, 1, 3, 0]</td>
</tr>
</tbody>
      </table>
<h5>입출력 예 설명</h5>

<p>classic 장르는 1,450회 재생되었으며, classic 노래는 다음과 같습니다.</p>

<ul>
<li>고유 번호 3: 800회 재생</li>
<li>고유 번호 0: 500회 재생</li>
<li>고유 번호 2: 150회 재생</li>
</ul>

<p>pop 장르는 3,100회 재생되었으며, pop 노래는 다음과 같습니다.</p>

<ul>
<li>고유 번호 4: 2,500회 재생</li>
<li>고유 번호 1: 600회 재생</li>
</ul>

<p>따라서 pop 장르의 [4, 1]번 노래를 먼저, classic 장르의 [3, 0]번 노래를 그다음에 수록합니다.</p>

<p>※ 공지 - 2019년 2월 28일 테스트케이스가 추가되었습니다.</p>
</div>
    </div>

In [77]:
'''
맨 먼저 만들어놓을 딕셔너리 
{장르: {전체 시간, 노래리스트: [{노래1 시간, 인덱스1}, {노래2 시간, 인덱스2} ...]}}

1. 장르 전체 시간 별로 sort
2. 노래 시간 별로 sort
3. 최대 2곡씩 answer에 append
'''
def solution(genres, plays):
    dic = {}

    for i in range(len(genres)):
        if not genres[i] in dic:
            dic[genres[i]] = {'total_time':0, 'songs':[]}

        dic[genres[i]]['songs'].append({'time':plays[i], 'idx':i})
        dic[genres[i]]['total_time'] += plays[i]
    print(dic)

    dic = sorted(dic.items(), key=lambda x: x[1]['total_time'], reverse=True)
    
    answer = []
    for i in range(len(dic)):
        song_list = dic[i][1]['songs']
        song_list = sorted(song_list, key=lambda x: x['time'], reverse=True)

        if len(song_list) >= 2:
            iteration = 2  
        else:
            iteration = len(song_list)

        for j in range(iteration):
            answer.append(song_list[j]['idx'])
            
    return answer


# defaultdict로 첨에 반복문 안써도 초기화 할 수 있는 방법이 없을까?